# End-to-End Demonstration: Canvas

*Goal*: run the Canvas Extractor and then upload the results into an `LMS` database.

## Software Requirements

1. Be sure to install Python 3.9; if you have multiple versions, make sure that the `python` command runs version 3.9.x. You can confirm your version by running `python --version` at a command prompt.
1. Microsoft SQL Server 2017 or 2019, in Windows or Linux.

## Getting Started

1. Confirm you have [poetry](https://python-poetry.org) installed (`poetry --version`).
1. Follow the [notebook instructions](README.md) to install dependencies used by this notebook.
1. Create an `LMS` database in SQL Server.
1. Please read through the notebook and fill in the two text boxes below before executing the entire notebook.

In [1]:
# Load some utilities
from IPython.display import display, Markdown
import ipywidgets as widgets

# Setup logging
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Prepare Input Data

Update the variables in the next block as needed. Below that, there will be are runtime text input boxes to provide the agency's Canvas URL and a Canvas access token. [Instructions for acquiring an access token](https://community.canvaslms.com/t5/Student-Guide/How-do-I-manage-API-access-tokens-as-a-student/ta-p/273).

In [2]:
LOG_LEVEL = "INFO"
OUTPUT_DIRECTORY = "c-data"
START_DATE="2021-02-01"
END_DATE="2021-07-28"
DB_ENGINE = "mssql"
DB_SERVER = "localhost"
DB_NAME = "LMS"
DB_PORT = 1433

In [3]:
def get_access_token(token, base_url):
    return token, base_url

token_widget = widgets.Text(
    description="Access Token"
)
base_url_widget = widgets.Text(
    description="Base URL"
)
result = widgets.interactive(get_access_token, token=token_widget, base_url=base_url_widget)
display(result)

interactive(children=(Text(value='', description='Access Token'), Text(value='', description='Base URL'), Outp…

In [5]:
TOKEN = result.children[0].value
CANVAS_BASE_URL = result.children[1].value

## Run the Canvas Extractor

In [8]:
from edfi_canvas_extractor.helpers.arg_parser import MainArguments as c_args
from edfi_canvas_extractor import extract_facade

arguments = c_args(
    base_url=CANVAS_BASE_URL,
    access_token=TOKEN,
    output_directory=OUTPUT_DIRECTORY,
    log_level=LOG_LEVEL,
    start_date=START_DATE,
    end_date=END_DATE
)

extract_facade.run(arguments)

INFO:edfi_canvas_extractor.extract_facade:Starting Ed-Fi LMS Canvas Extractor
INFO:edfi_canvas_extractor.extract_facade:Extracting Courses from Canvas API
INFO:edfi_canvas_extractor.api.courses:Pulling course data
INFO:canvasapi.requester:Request: GET https://YOURSCHOOL.instructure.com/api/v1/accounts
INFO:canvasapi.requester:Response: GET https://YOURSCHOOL.instructure.com/api/v1/accounts 200
INFO:canvasapi.requester:Request: GET https://YOURSCHOOL.instructure.com/api/v1/accounts/1/courses
INFO:canvasapi.requester:Response: GET https://YOURSCHOOL.instructure.com/api/v1/accounts/1/courses 200
INFO:canvasapi.requester:Request: GET https://YOURSCHOOL.instructure.com/api/v1/accounts/1/courses?ends_after=2021-02-01&starts_before=2021-07-28&state%5B%5D=available&state%5B%5D=completed&page=2&per_page=100
INFO:canvasapi.requester:Response: GET https://YOURSCHOOL.instructure.com/api/v1/accounts/1/courses?ends_after=2021-02-01&starts_before=2021-07-28&state%5B%5D=available&state%5B%5D=completed

## Run the Learning Management System Data Store Loader (LMS-DS-Loader)

The default setup below uses Windows integrated security. For username/password security, please review the commented-out code.

In [9]:
from edfi_lms_ds_loader.helpers.argparser import MainArguments as lms_args
from edfi_lms_ds_loader import loader_facade

arguments = lms_args(
    OUTPUT_DIRECTORY,
    DB_ENGINE,
    LOG_LEVEL
)
arguments.set_connection_string_using_integrated_security(
    DB_SERVER,
    DB_PORT,
    DB_NAME,
)
# For password auth, comment out the line above and uncomment this one:
# arguments.set_connection_string(
#     DB_SERVER,
#     DB_PORT,
#     DB_NAME,
#     USERNAME,
#     PASSWORD,
# )

loader_facade.run_loader(arguments)

INFO:edfi_lms_ds_loader.loader_facade:Begin loading files into the LMS Data Store (DS)...
INFO:edfi_lms_ds_loader.migrator:Begin database auto-migration...
INFO:edfi_lms_ds_loader.migrator:Done with database auto-migration.
INFO:edfi_lms_ds_loader.df_to_db:Uploading LMSUser file ...
INFO:edfi_lms_ds_loader.df_to_db:Done with LMSUser file.
INFO:edfi_lms_ds_loader.df_to_db:Uploading LMSSection file ...
INFO:edfi_lms_ds_loader.df_to_db:Done with LMSSection file.
INFO:edfi_lms_ds_loader.loader_facade:No sections loaded. Skipping assignments.
INFO:edfi_lms_ds_loader.loader_facade:No assignments loaded. Skipping assignment submissions.
INFO:edfi_lms_ds_loader.loader_facade:No sections loaded. Skipping section associations.
INFO:edfi_lms_ds_loader.loader_facade:No sections loaded. Skipping section associations.
INFO:edfi_lms_ds_loader.loader_facade:No sections loaded. Skipping section associations.
INFO:edfi_lms_ds_loader.loader_facade:Done loading files into the LMS Data Store.
